In [ ]:
# deps
!pip install opencv-python easyocr numpy matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr

# === 1) Load image ===
img = cv2.imread("pvi_cv08_spz.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# === 2) Threshold and morphological processing ===
# Use adaptive threshold for robustness
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 41, 15)

# Morphological closing to remove text and fill plate area
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# === 3) Harris corner detection ===
gray_float = np.float32(gray)
dst = cv2.cornerHarris(gray_float, blockSize=5, ksize=3, k=0.04)
dst = cv2.dilate(dst, None)

# Threshold for selecting strong corners
corners = np.argwhere(dst > 0.01 * dst.max())

# Draw corners
img_harris = img.copy()
for y, x in corners:
    cv2.circle(img_harris, (x, y), 4, (0, 0, 255), -1)

# === 4) Find rotation angle from corner points ===
# Use PCA to estimate main axis orientation
pts = np.column_stack(np.where(morph > 0))
mean, eigenvectors = cv2.PCACompute(pts.astype(np.float32), mean=None)
angle = np.degrees(np.arctan2(eigenvectors[0, 1], eigenvectors[0, 0]))

# === 5) Rotate image to horizontal ===
(h, w) = img.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

# === 6) OCR using EasyOCR ===
reader = easyocr.Reader(['en'])
result = reader.readtext(rotated)

# Combine detected text
text_result = " ".join([res[1] for res in result])

# === 7) Write text above plate ===
output = rotated.copy()
cv2.putText(output, text_result, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

# === 8) Visualization ===
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
plt.title("Orig. Im.")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 2)
plt.title("Bin. Im.")
plt.imshow(morph, cmap='jet')

plt.subplot(2, 2, 3)
plt.title("Im. + Harris")
plt.imshow(cv2.cvtColor(img_harris, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 4)
plt.title("Final Result (rotated + OCR)")
plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.show()

print("Recognized text:", text_result)
